<a href="https://colab.research.google.com/github/GAffig/DS4A-/blob/main/Linear_Regression_Model_Years_of_Life_Lost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Google Drive Install and Import

#mount Google Drive to access files
from google.colab import drive
drive.mount('/content/drive')

#Install Requirements via Text File
!pip install -r /content/drive/MyDrive/DS4A_Team_25/requirements.txt

#Import the packages for cleaning, analysis, and visualization 

import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy.stats as stats
import pandas_profiling as pp
from pandas_profiling import ProfileReport
from pandasql import sqldf
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from statsmodels.formula.api import ols
import statsmodels.api as smf




#Read the data file into a dataframe
with open('/content/drive/MyDrive/DS4A_Team_25/data/mass_killing_incidents_public.csv') as f:
    mki=pd.read_csv(f, delimiter=',')
with open('/content/drive/MyDrive/DS4A_Team_25/data/mass_killing_offenders_public.csv') as f:
    mko=pd.read_csv(f, delimiter=',')
with open('/content/drive/MyDrive/DS4A_Team_25/data/mass_killing_victims_public.csv') as f:
    mkv=pd.read_csv(f, delimiter=',')
with open('/content/drive/MyDrive/DS4A_Team_25/data/mass_killing_weapons_public.csv') as f:
    mkw=pd.read_csv(f, delimiter=',')


Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     / 22.1 MB 24.0 MB/s 0:00:02
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.0/188.0 KB 4.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 35.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 97.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.7/102.7 KB 10.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 679.5/679.5 KB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 73.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 KB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

<ipython-input-1-af228786fe21>:18: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  import pandas_profiling as pp


In [ ]:
#Merging Mass Killing Incident Table + Victim Table + Offender Table

query0 = """ 
            SELECT mki.incident_id, date as incidentdate, city, state, num_offenders,
            num_victims_killed, num_victims_injured, firstcod, secondcod,
            type, situation_type, location_type, location, longitude,
            latitude, narrative, victim_id, mkv.age as v_age, mkv.race as v_race,
            mkv.sex as v_sex, vorelationship, offender_id,
              CASE 
                WHEN mko.age < 0 THEN NULL
                ELSE mko.age
                END AS o_age, 
              mko.race as o_race, mko.sex as o_sex, suicide, outcome
            FROM mki
              LEFT JOIN mkv
              ON mki.incident_id = mkv.incident_id
              LEFT JOIN mko
              ON mki.incident_id = mko.incident_id          
        """
# Query execution and convert to dataframe
mki_mkv_mko = sqldf(query0).copy()

#Cleaning Dataframe
#mki_mkv_mkw['incidentdate'] = pd.to_datetime(mki_mkv_mkw['date'])
#mki_mkv_mkw = mki_mkv_mkw.drop(columns=['date'])
#mki_mkv_mkw['num_victims_injured'] = mki_mkv_mkw['num_victims_injured'].astype(int)
mki_mkv_mko['incident_id'] = mki_mkv_mko['incident_id'].astype(str)
mki_mkv_mko['victim_id'] = mki_mkv_mko['victim_id'].astype(str)
mki_mkv_mko['offender_id'] = mki_mkv_mko['offender_id'].astype(str)
#mki_mkv_mkw['victim_age'] = mki_mkv_mkw['victim_age'].astype(int)
mki_mkv_mko = mki_mkv_mko.drop_duplicates()
mki_mkv_mko.sample(3)

,incident_id,incidentdate,city,state,num_offenders,num_victims_killed,num_victims_injured,firstcod,secondcod,type,...,v_age,v_race,v_sex,vorelationship,offender_id,o_age,o_race,o_sex,suicide,outcome
999,373,2018-03-26,Westport,CA,2,6,0.0,Vehicle crash,None,Family,...,15.0,Black,Male,Child or stepchild,457.0,38.0,White,Female,1.0,Suicide
1465,300,2016-06-12,Orlando,FL,1,49,53.0,Shooting,None,Public,...,26.0,Hispanic/Latino,Male,Random bystander/stranger,371.0,29.0,Asian/Pacific Islander,Male,0.0,Killed
480,447,2020-11-30,Palmer,AK,1,4,0.0,Shooting,None,Family,...,18.0,American Indian,Male,Cousin,560.0,18.0,American Indian,Male,0.0,Arrested/Pending trial


In [ ]:
# Importing Life Expectancy Data for the U.S. by Gender
with open('/content/drive/MyDrive/DS4A_Team_25/data/le/U.S._State_Life_Expectancy_by_Sex__2020.csv') as f:
  le=pd.read_csv(f, delimiter=',')

In [ ]:
# US State to Abbreviations
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}

abbv_to_states = dict(zip(us_state_to_abbrev.values(), us_state_to_abbrev.keys()))

states_to_regions = {
    'Washington': 'West', 'Oregon': 'West', 'California': 'West', 'Nevada': 'West',
    'Idaho': 'West', 'Montana': 'West', 'Wyoming': 'West', 'Utah': 'West',
    'Colorado': 'West', 'Alaska': 'West', 'Hawaii': 'West', 'Maine': 'Northeast',
    'Vermont': 'Northeast', 'New York': 'Northeast', 'New Hampshire': 'Northeast',
    'Massachusetts': 'Northeast', 'Rhode Island': 'Northeast', 'Connecticut': 'Northeast',
    'New Jersey': 'Northeast', 'Pennsylvania': 'Northeast', 'North Dakota': 'Midwest',
    'South Dakota': 'Midwest', 'Nebraska': 'Midwest', 'Kansas': 'Midwest',
    'Minnesota': 'Midwest', 'Iowa': 'Midwest', 'Missouri': 'Midwest', 'Wisconsin': 'Midwest',
    'Illinois': 'Midwest', 'Michigan': 'Midwest', 'Indiana': 'Midwest', 'Ohio': 'Midwest',
    'West Virginia': 'South', 'District of Columbia': 'South', 'Maryland': 'South',
    'Virginia': 'South', 'Kentucky': 'South', 'Tennessee': 'South', 'North Carolina': 'South',
    'Mississippi': 'South', 'Arkansas': 'South', 'Louisiana': 'South', 'Alabama': 'South',
    'Georgia': 'South', 'South Carolina': 'South', 'Florida': 'South', 'Delaware': 'South',
    'Arizona': 'Southwest', 'New Mexico': 'Southwest', 'Oklahoma': 'Southwest',
    'Texas': 'Southwest'}

regions_to_states = dict(zip(states_to_regions.values(), states_to_regions.keys()))

month_season = {
    1:'winter', 2:'winter',3:'spring',4:'spring',5:'spring',6:'summer',
    7:'summer',8:'summer',9:'fall',10:'fall',11:'fall',12:'winter'
}

In [ ]:
#2020 Life Expectancy by State and Year
le_state = le.copy()
le_state['region'] = le_state['State'].map( states_to_regions)
pd.set_option('display.max_rows', 200)
le_state = le_state.replace({"State": us_state_to_abbrev})
le_state.head(5)

,State,Sex,LE,SE,Quartile,region
0,AL,Total,73.2,0.067,71.9 - 75.3,South
1,AK,Total,76.6,0.176,75.4 - 76.8,West
2,AZ,Total,76.3,0.055,75.4 - 76.8,Southwest
3,AR,Total,73.8,0.086,71.9 - 75.3,South
4,CA,Total,79.0,0.022,78.1 - 80.7,West


In [ ]:
# Merging of 2020 State Life Expectancy and Mass Killing and Incident Tables
# Query definition
query1 = """ 
            SELECT *,le.le - v_age as yrs_life_lost
            FROM mki_mkv_mko
              LEFT JOIN le_state as le
              ON mki_mkv_mko.state = le.state and mki_mkv_mko.v_sex = le.sex
       
        """
# Query execution and convert to dataframe
mk_vo_le = sqldf(query1).copy()
mk_vo_le.replace(to_replace=[None], value=np.nan, inplace=True)
mk_vo_le['incidentdate'] = pd.to_datetime(mk_vo_le['incidentdate'],format="%Y-%m-%d")
mk_vo_le['month'] = mk_vo_le['incidentdate'].dt.month.astype('category')
mk_vo_le['year'] = mk_vo_le['incidentdate'].dt.year
mk_vo_le['season'] = mk_vo_le['month'].map(month_season)
mk_vo_le['secondcod'] = mk_vo_le['secondcod'].fillna('None')
mk_vo_le['first_second_cod'] = mk_vo_le['firstcod'] + '-' + mk_vo_le['secondcod']
mk_vo_le = mk_vo_le[mk_vo_le['year'] <= 2022].copy()
mk_vo_le['season_month'] = mk_vo_le['season'] + '-' + mk_vo_le['month'].astype(str)
mk_vo_le['num_offenders'] = mk_vo_le['num_offenders'].astype(int)
mk_vo_le['region_state'] = mk_vo_le['region'] + '-' + mk_vo_le['state']
mk_vo_le['o_race_sex'] = mk_vo_le['o_race'] + '-' + mk_vo_le['o_sex']
mk_vo_le['v_race_sex'] = mk_vo_le['v_race'] + '-' + mk_vo_le['v_sex']
mk_vo_le['city_state'] = mk_vo_le['city'] + '-' + mk_vo_le['state']
mk_vo_le = mk_vo_le.drop_duplicates()
mk_vo_le = mk_vo_le.dropna().reset_index()
mk_vo_le = mk_vo_le.drop(columns=["State","Sex","Quartile","SE","LE","index","year","latitude","longitude","narrative",]) #"firstcod","secondcod"


# function for age range of offender
def age_range(x):
    if x < 21:
        return 'Under 21'
    elif x >= 21 and x <= 30:
        return '21-30 Years'
    elif x >= 31 and x <= 50:
        return '31-50 Years'
    else:
        return '51+ Years'
mk_vo_le['o_age_range'] = mk_vo_le['o_age'].apply(age_range)


# One-hot encode the 'region' variable
# mki_all_le = pd.get_dummies(mki_all_le, columns=['season'])

pd.set_option('display.max_columns', None)
mk_vo_le.sample(3)

,incident_id,incidentdate,city,state,num_offenders,num_victims_killed,num_victims_injured,firstcod,secondcod,type,situation_type,location_type,location,victim_id,v_age,v_race,v_sex,vorelationship,offender_id,o_age,o_race,o_sex,suicide,outcome,region,yrs_life_lost,month,season,first_second_cod,season_month,region_state,o_race_sex,v_race_sex,city_state,o_age_range
1297,270,2015-06-17,Charleston,SC,1,9,1.0,Shooting,None,Public,Hate,House of Worship,House of worship,1349,49.0,Black,Female,Random bystander/stranger,336.0,21.0,White,Male,0.0,Death sentence,South,29.0,6,summer,Shooting-None,summer-6,South-SC,White-Male,Black-Female,Charleston-SC,21-30 Years
1312,268,2015-06-07,Deer Lodge,MT,1,4,0.0,Shooting,None,Family,Interpersonal conflict,Residence/Other shelter,Residence,1339,4.0,Other,Male,Child or stepchild,333.0,59.0,White,Male,1.0,Suicide,West,70.2,6,summer,Shooting-None,summer-6,West-MT,White-Male,Other-Male,Deer Lodge-MT,51+ Years
1743,136,2011-12-15,Gargatha,VA,1,4,0.0,Shooting,None,Family,Interpersonal conflict,Residence/Other shelter,Residence,710,20.0,Hispanic/Latino,Female,Roommate,178.0,37.0,Hispanic/Latino,Male,1.0,Suicide,South,60.1,12,winter,Shooting-None,winter-12,South-VA,Hispanic/Latino-Male,Hispanic/Latino-Female,Gargatha-VA,31-50 Years


In [ ]:
mk_vo_le.columns

Index(['incident_id', 'incidentdate', 'city', 'state', 'num_offenders',
       'num_victims_killed', 'num_victims_injured', 'firstcod', 'secondcod',
       'type', 'situation_type', 'location_type', 'location', 'victim_id',
       'v_age', 'v_race', 'v_sex', 'vorelationship', 'offender_id', 'o_age',
       'o_race', 'o_sex', 'suicide', 'outcome', 'region', 'yrs_life_lost',
       'month', 'season', 'first_second_cod', 'season_month', 'region_state',
       'o_race_sex', 'v_race_sex', 'city_state', 'o_age_range'],
      dtype='object')

In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from statsmodels.formula.api import ols

# Load the data
df_reg = mk_vo_le.copy()

skip_columns = ['incident_id', 'incidentdate', 'num_offenders',
       'num_victims_killed', 'num_victims_injured','victim_id','offender_id','yrs_life_lost', 'year']
encoders = {}

for column in df_reg.select_dtypes(include=['object']).columns:
    if column in skip_columns:
        continue
    if df_reg[column].dtype == 'object':
        le = LabelEncoder()
        df_reg[column] = le.fit_transform(df_reg[column])
        encoders[column] = le

inverse_mapping = {}

for column, le in encoders.items():
   inverse_mapping[column] = dict(zip(le.transform(le.classes_), le.classes_))

In [ ]:
mapped_model_values = pd.DataFrame.from_dict(inverse_mapping, orient='columns').reset_index()
#mapped_model_values.columns
mapped_model_values_pivot = mapped_model_values.melt(id_vars=['index'],value_vars=['index', 'type', 'situation_type', 'vorelationship','season_month','o_race_sex',
       'v_race_sex', 'o_age_range','city_state'])
mapped_model_values_pivot.head(3)

,index,variable,value
0,0,type,Family
1,1,type,Felony
2,2,type,Other


In [ ]:
# Perform linear regression
#formula = 'yrs_life_lost ~  C(city_state) + C(vorelationship) + C(v_race_sex) + C(situation_type) + C(type) + C(season_month) + C(o_age_range)'
formula = 'yrs_life_lost ~ C(city_state) + C(vorelationship) + C(season_month) + C(v_race_sex) + C(situation_type) + C(type) + C(o_age_range)'


model = ols(formula, df_reg,scale=True, exclude=None).fit()

# Execute ANOVA Analysis
aov_table = smf.stats.anova_lm(model, typ=2)
aov_table_df = pd.DataFrame(aov_table)

# Print summary of ANOVA Analysis
aov_table_df['Significant']=aov_table_df['PR(>F)'] < 0.05
aov_table_df



/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:127: ValueWarning: unknown kwargs ['scale', 'exclude']
  warnings.warn(msg, ValueWarning)


,sum_sq,df,F,PR(>F),Significant
C(city_state),291711.637512,365.0,4.908847,2.476104e-126,True
C(vorelationship),157397.059172,28.0,34.526868,7.008495e-154,True
C(season_month),8425.756643,11.0,4.704731,3.653194e-07,True
C(v_race_sex),20096.747466,11.0,11.221520,1.378981e-20,True
C(situation_type),10458.535187,13.0,4.941356,1.213847e-08,True
C(type),3906.560401,6.0,3.999097,5.459384e-04,True
C(o_age_range),2218.782849,3.0,4.542680,3.516043e-03,True
Residual,390581.483153,2399.0,NaN,NaN,False


In [ ]:
# Create summary of regeression model
cond = model.condition_number
r2 = model.rsquared
r2a = model.rsquared_adj
f1 = model.fvalue
f1_p = model.f_pvalue
summary = model.summary2()

# Print summary of regression model
print("Condition No:", cond)
print("R-Squared:",r2)
print("Adjusted R-Squared:",r2a)
print("F1 Statistic:",f1)
print("F1 Statistic P-Value:",f1)
print("Model Variables:",model.params)

#print(summary)

Condition No: 710.3868145950088
R-Squared: 0.6795165811004162
Adjusted R-Squared: 0.6211375673200419
F1 Statistic: 11.639740672166914
F1 Statistic P-Value: 11.639740672166914
Model Variables: Intercept              47.076746
C(city_state)[T.1]      1.024337
C(city_state)[T.2]    -56.291722
C(city_state)[T.3]    -10.315072
C(city_state)[T.4]    -20.527702
                         ...    
C(type)[T.5]           20.750667
C(type)[T.6]           33.357430
C(o_age_range)[T.1]    -3.953346
C(o_age_range)[T.2]    -6.071714
C(o_age_range)[T.3]     1.505787
Length: 438, dtype: float64


In [ ]:
#Model summary
print(summary)

                     Results: Ordinary least squares
Model:                 OLS                Adj. R-squared:       0.621     
Dependent Variable:    yrs_life_lost      AIC:                  22898.9686
Date:                  2023-02-05 18:44   BIC:                  25505.2887
No. Observations:      2837               Log-Likelihood:       -11011.   
Df Model:              437                F-statistic:          11.64     
Df Residuals:          2399               Prob (F-statistic):   0.00      
R-squared:             0.680              Scale:                162.81    
--------------------------------------------------------------------------
                         Coef.   Std.Err.    t    P>|t|   [0.025   0.975] 
--------------------------------------------------------------------------
Intercept                47.0767  13.5936  3.4632 0.0005  20.4204  73.7331
C(city_state)[T.1]        1.0243   9.6799  0.1058 0.9157 -17.9575  20.0061
C(city_state)[T.2]      -56.2917  10.3516 -5.43